In [10]:
# of ⁨5⁩
# Odwzorowanie Gaussa-Krügera: układy współrzędnych
# płaskich stosowanych w Polsce
# Wybrane Zagadnienia Geodezji Wyższej
# Maciej Grzymała
# maciej.grzymala@pw.edu.pl
# Wydział Geodezji i Kartografii, Politechnika Warszawska
# Warszawa, 2023
# 1 Algorytm przeliczenia współrzędnych φ i λ na płaszczyznę
# odwzorowania Gaussa-Krügera
# Oznaczenia
# • a, b – dłuższa i krótsza półoś elipsoidy;
# • e, e′ – pierwszy mimośród, drugi mimośród;
# • φ, λ – szerokość i długość geodezyjna;
# • xgk, ygk – współrzędne prostokątne lokalne na płaszczyźnie Gaussa-Krügera;
# • λ0 – długość geodezyjna południka zerowego;
# • σ – długość łuku południka;
# 1.1 Przeliczenie wprost: (xgk, ygk) = f (φ, λ)
# Dane
# • φ, λ – współrzędne geodezyjne punktu;
# • λ0 – długość geodezyjna południka zerowego;
# • a, e2 – parametry elipsoidy;
# Szukane
# • xgk, ygk – współrzędne prostokątne lokalne na płaszczyźnie Gaussa-Krügera;
# Algorytm
# 1. Parametry elipsoidy:
# b2 = a2(1 − e2); e′2 = a2 − b2
# b2 ;
# 1
# 2. Wielkości pomocnicze:
# ∆λ = λ − λ0; t = tan φ; η2 = e′2 · cos2 φ; N = a
# √1 − e2 sin2 φ ;
# 3. Długość łuku południka:
# σ = a · (A0φ − A2 sin 2φ + A4 sin 4φ − A6 sin 6φ + . . . );
# gdzie:
# A0 = 1− e2
# 4 − 3e4
# 64 − 5e6
# 256 ; A2 = 3
# 8
# (
# e2 + e4
# 4 + 15e6
# 128
# )
# ; A4 = 15
# 256
# (
# e4 + 3e6
# 4
# )
# ; A6 = 35e6
# 3072 ;
# 4. Współrzędne prostokątne lokalne na płaszczyźnie Gaussa-Krügera:
# xgk = σ + ∆λ2
# 2 N sin φ cos φ
# {
# 1 + ∆λ2
# 12 cos2 φ ·
# (
# 5 − t2 + 9η2 + 4η4)
# + ∆λ4
# 360 cos4 φ ·
# (
# 61 − 58t2 + t4 + 270η2 − 330η2t2)
# + . . .
# }
# ;
# ygk = ∆λN cos φ ·
# {
# 1 + ∆λ2
# 6 cos2 φ ·
# (
# 1 − t2 + η2)
# + ∆λ4
# 120 cos4 φ
# (
# 5 − 18t2 + t4 + 14η2 − 58η2t2)
# + . . .
# }
# ;
# 1.2 Przeliczenie odwrotne: (φ, λ) = f (xgk, ygk)
# Dane
# • xgk, ygk – współrzędne prostokątne lokalne na płaszczyźnie Gaussa-Krügera;
# • λ0 – długość geodezyjna południka zerowego;
# • a, e2 – parametry elipsoidy;
# Szukane
# • φ, λ – współrzędne geodezyjne punktu;
# Algorytm
# 1. Iteracyjne wyznaczenie pierwszego przybliżenia szerokości geodezyjnej φ1:
# φi=0
# 1 = xgk
# aA0
# ; σi = f (φi
# 1) – na podstawie wzoru pkt. 2.3.3;
# φi
# 1 = φi−1
# 1 + xgk − σi−1
# aA0
# ; Warunek zakończenia iteracji: |φi
# 1 − φi−1
# 1 | < 0”.000001
# 2. Wielkości pomocnicze (obliczone w funkcji φ1):
# N1; M1; t1; η1;
# 2
# 3. Współrzędne geodezyjne:
# φ = φ1 − y2
# gkt1
# 2M1N1
# ·
# {
# 1 − y2
# gk
# 12N 2
# 1
# ·
# (
# 5 + 3t2
# 1 + η2
# 1 − 9η2
# 1 t2
# 1 − 4η4
# 1
# )
# + y4
# gk
# 360N 4
# 1
# ·
# (
# 61 + 90t2
# 1 + 45t4
# 1
# )
# + . . .
# }
# ;
# λ = λ0 + ygk
# N1 cos φ1
# ·
# {
# 1 − y2
# gk
# 6N 2
# 1
# ·
# (
# 1 + 2t2
# 1 + η2
# 1
# )
# + y4
# gk
# 120N 4
# 1
# ·
# (
# 5 + 28t2
# 1 + 24t4
# 1 + 6η2
# 1 + 8η2
# 1 t2
# 1
# )
# + . . .
# }
# ;
# 2 Układ współrzędnych płaskich PL-2000
# Układ współrzędnych prostokątnych płaskich 2000 oparty jest na odwzorowaniu G-K. Obszar
# Polski podzielony jest na 4 pasy południkowe, o szerokości 3◦ każdy (południki osiowe 15◦, 18◦,
# 21◦, 24◦), oznaczone odpowiednio numerami 5, 6, 7 i 8. Współczynnik zniekształcenia skali w
# południku osiowym każdego pasa równy jest 0.999923 (m0 = 0.999923). Układ przeznaczony do
# opracowań w skalach większych od 1:10000, a w szczególności mapy zasadniczej.
# x2000 = m0 · xgk (1)
# y2000 = m0 · ygk + nr · 1 000 000 + 500 000 (2)
# 3 Układ współrzędnych płaskich PL-1992
# Układ współrzędnych prostokątnych płaskich 1992 oparty jest na odwzorowaniu G-K. Obszar
# Polski objęty jest jednym pasem południkowym o szerokości 10◦ (południk osiowy 19◦ długości
# geodezyjnej wschodniej). Współczynnik zniekształcenia skali w południku osiowym każdego pasa
# równy jest 0.9993 (m0 = 0.9993). Układ przeznaczony do opracowań w skali 1:10000 i skalach
# mniejszych oraz do tworzenia Topograficznej Bazy Danych.
# x1992 = m0 · xgk − 5 300 000 (3)
# y1992 = m0 · ygk + 500 000 (4)
# 4 Zadanie 1. Przeliczenie współrzędnych do układów PL-1992
# i PL-2000
# Mając dane współrzędne geodezyjne φ, λ czterech punktów, z ćwiczenia nr 3, wyznaczone
# na podstawie zadania wprost przeniesienia współrzędnych na elipsoidzie, metodą Kivioji, oblicz
# współrzędne płaskie tych punktów w układach PL-1992 oraz PL-2000. Wyznaczając współrzędne
# punktów w układzie PL-2000 należy pamiętać o odpowiedniej strefie odwzorowania Gaussa-
# Krügera. W przypadku, gdy nie wszystkie z danych punktów będą leżeć wewnątrz tego samego
# 3-stopniowego pasa odwzorowania G-K, należy przyjąć strefę odpowiednią dla punktu nr 1 (tak,
# aby współrzędne wszystkich 4 punktów wyznaczyć w tym samym układzie współrzędnych pła-
# skich).
# 3


Mając dane współrzędne geodezyjne φ, λ czterech punktów, z ćwiczenia nr 3, wyznaczone
na podstawie zadania wprost przeniesienia współrzędnych na elipsoidzie, metodą Kivioji, oblicz
współrzędne płaskie tych punktów w układach PL-1992 oraz PL-2000. Wyznaczając współrzędne
punktów w układzie PL-2000 należy pamiętać o odpowiedniej strefie odwzorowania Gaussa-
Krügera. W przypadku, gdy nie wszystkie z danych punktów będą leżeć wewnątrz tego samego
3-stopniowego pasa odwzorowania G-K, należy przyjąć strefę odpowiednią dla punktu nr 1 (tak,
aby współrzędne wszystkich 4 punktów wyznaczyć w tym samym układzie współrzędnych pła-
skich).

In [11]:
import numpy as np
from pyproj import Transformer, CRS, Geod

# phi = 52.005893
# lam = 18.571495
# input_proj = CRS.from_epsg(2180)
# output_proj = CRS.from_epsg(4326)


# x, y = Transformer.from_proj(input_proj, output_proj).transform(phi, lam)
# print(x, y)

nr = 2
m0 = 0.999923
phi1 = 50 + nr*15/60
lam1 = 18 + nr*15/60

geod = Geod(ellps='GRS80')

d12 = 40e3
a12 = 0
d23 = 100e3
a23 = 90
d34 = 40e3
a34 = 180

p1 = (lam1,phi1,0)
p2 = geod.fwd(lam1,phi1,a12,d12)
p3 = geod.fwd(p2[0],p2[1],a23,d23)
p4 = geod.fwd(p3[0],p3[1],a34,d34)


# Transformacja współrzędnych geodezyjnych na płaszczyźnie Gaussa-Krügera

input_proj = CRS.from_epsg(4326)
output_proj = CRS.from_epsg(2180)
philam_2000_transformer = Transformer.from_proj(input_proj, output_proj)


pl2000_1 = philam_2000_transformer.transform(p1[1], p1[0])
pl2000_2 = philam_2000_transformer.transform(p2[1], p2[0])
pl2000_3 = philam_2000_transformer.transform(p3[1], p3[0])
pl2000_4 = philam_2000_transformer.transform(p4[1], p4[0])

pl2000_points = [pl2000_1, pl2000_2, pl2000_3, pl2000_4]


pl1992_1 = (pl2000_1[0] - 5300000, pl2000_1[1] - nr*1000000)
pl1992_2 = (pl2000_2[0] - 5300000, pl2000_2[1] - nr*1000000)
pl1992_3 = (pl2000_3[0] - 5300000, pl2000_3[1] - nr*1000000)
pl1992_4 = (pl2000_4[0] - 5300000, pl2000_4[1] - nr*1000000)

pl1992_points = [pl1992_1, pl1992_2, pl1992_3, pl1992_4]

print('Numer punktu\tx1992\t\t\ty1992\t\t\tx2000\t\ty2000')
for i in range(len(pl1992_points)):
    print(f'{i+1}\t\t{pl1992_points[i][0]:.3f}\t\t{pl1992_points[i][1]:.3f}\t\t{pl2000_points[i][0]:.3f}\t\t{pl2000_points[i][1]:.3f}')


Numer punktu	x1992			y1992			x2000		y2000
1		-5007334.189		-1535449.803		292665.811		464550.197
2		-4967362.487		-1535179.945		332637.513		464820.055
3		-4968038.862		-1435250.943		331961.138		564749.057
4		-5008009.854		-1434754.394		291990.146		565245.606


Obliczenie pola powierzchni "trapezu"na płaszczyźnie Pole powierzchni figury na
płaszczyźnie wyrazić można stosując tzw. wzory Gaussa:
2P =
n∑
i=1
xi(yi+1 − yi−1) (14)
dla kontroli:
−2P =
n∑
i=1
yi(xi+1 − xi−1) (15)
(równania dotyczą przypadku, kiedy punkty numerowane są zgodnie z ruchem wskazówek zegara.
W sytuacji odwrotnej, należy wziąć pod uwagę wartości bezwzględne)
Należy obliczyć pole powierzchni danego "trapezu"na płaszczyźnie układów PL-2000 oraz
PL-1992. Posługując się np. biblioteką pyproj języka python, należy również przeliczyć współ-
rzędne punktów do układu PL-LAEA – układu równopolowego (kody EPSG układów współ-
rzędnych płaskich można znaleźć na stronie: https://epsg.org/home.html). Należy zatem
policzyć dodatkowo pole powierzchni figury na podstawie współrzędnych płaskich w tym ukła-
dzie, sprawdzając tym samym wiernopolowość odwzorowania (pole powierzchni należy porównać
z polem figury na elipsoidzie, wyznaczonym w zadaniu 3).

In [12]:
def pole_wielokata(points: list) -> float:
    p = 0
    for i in range(1, len(points) - 1):
        p += points[i][0] * (points[i+1][1] - points[i-1][1])
    # last element
    p += points[-1][0] * (points[0][1] - points[-2][1])
    # first element
    p += points[0][0] * (points[1][1] - points[-1][1])
    return abs(p/2)


pole_trapezu_1992 = pole_wielokata(pl1992_points)
pole_trapezu_2000 = pole_wielokata(pl2000_points)

# Układ współrzędnych PL-LAEA
# https://www.asgeupos.pl/webpg/graph/img/_news/00202/2.1.pdf slajd 33
# https://proj.org/operations/projections/laea.html
# • równopolowe, azymutalne
# odwzorowanie Lamberta,
# • początek układu: φ=52ºN i λ=10ºE,
# • współrzędne płaskie:
# x= 3210000 m,
# y= 4321000 m


pl_laea = CRS.from_proj4('+proj=laea +lat_0=52 +lon_0=10 +x_0=3210000 +y_0=4321000 +ellps=GRS80 +units=m +no_defs')
philam_laea_transformer = Transformer.from_proj(input_proj, pl_laea)

pl_laea_1 = philam_laea_transformer.transform(p1[1], p1[0])
pl_laea_2 = philam_laea_transformer.transform(p2[1], p2[0])
pl_laea_3 = philam_laea_transformer.transform(p3[1], p3[0])
pl_laea_4 = philam_laea_transformer.transform(p4[1], p4[0])

pl_laea_points = [pl_laea_1, pl_laea_2, pl_laea_3, pl_laea_4]

pole_trapezu_laea = pole_wielokata(pl_laea_points)

print(f'Pole trapezu w układzie PL-1992: {pole_trapezu_1992:.2f} m2')
print(f'Pole trapezu w układzie PL-2000: {pole_trapezu_2000:.2f} m2')
print(f'Pole trapezu w układzie PL-LAEA: {pole_trapezu_laea:.2f} m2')

Pole trapezu w układzie PL-1992: 4009537314.643 m2
Pole trapezu w układzie PL-2000: 4009537314.643 m2
Pole trapezu w układzie PL-LAEA: 4014988079.565 m2
